In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME= os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [5]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
graph

In [18]:
movie_query = """
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/refs/heads/main/movies/movies_small.csv' as row 

MERGE (m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person{name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person{name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre{name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))

"""

In [19]:
graph.query(movie_query)

[]

In [27]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [36]:
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key="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",model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f18c3bd4e90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f18c3bd5d30>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [37]:
llm.invoke("hey bro")

AIMessage(content="What's up, dude?  🤙  What can I do for you today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 11, 'total_tokens': 32, 'completion_time': 0.038181818, 'prompt_time': 0.00012217, 'queue_time': 0.01322965, 'total_time': 0.038303988}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9b7d95a0-0e63-4af2-b4cd-75771a4acb98-0', usage_metadata={'input_tokens': 11, 'output_tokens': 21, 'total_tokens': 32})

In [38]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(graph=graph,llm=llm,verbose=True,allow_dangerous_requests=True)

In [40]:
res = chain.invoke({"query":"how many artists are there"})
res



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n:Artist)
RETURN count(n) AS artistCount 



Failed to read from defunct connection ResolvedIPv6Address(('64:ff9b::227e:406e', 7687, 0, 0)) (ResolvedIPv6Address(('64:ff9b::227e:406e', 7687, 0, 0)))
Failed to read from defunct connection IPv4Address(('0b833f28.databases.neo4j.io', 7687)) (ResolvedIPv6Address(('64:ff9b::227e:406e', 7687, 0, 0)))
Transaction failed and will be retried in 1.1308128809959856s (Failed to read from defunct connection IPv4Address(('0b833f28.databases.neo4j.io', 7687)) (ResolvedIPv6Address(('64:ff9b::227e:406e', 7687, 0, 0))))
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: Artist)} {position: line: 2, column: 10, offset: 16} for query: 'cyph

Full Context:
[{'artistCount': 0}]

> Finished chain.


{'query': 'how many artists are there', 'result': 'There are 0 artists. \n'}